# Проект: классификация

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from  sklearn.ensemble import IsolationForest
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing  import LabelEncoder
from sklearn import linear_model 
from sklearn import tree 
from sklearn import ensemble 
from sklearn import metrics 
from sklearn import preprocessing 
from sklearn.model_selection import train_test_split 
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
import optuna
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, f1_score


import plotly
import plotly.express as px
import plotly.io as pio
#pio.renderers.default = "notebook_connected"
#pio.renderers.default = "notebook"  # для интерактивных графиков


In [ ]:
#!pip install --upgrade plotly

## Часть 1. Знакомство с данными, обработка пропусков и выбросов

### Задание 1

In [ ]:
df = pd.read_csv('data/bank_fin.csv', sep = ';')

In [ ]:
# исследуйте данные на предмет пропусков. Где есть пропущенные значения? Сколько их?
# ваш код
missing_values = df.isnull().sum().sort_values()

# Фильтруем только ненулевые значения
non_zero_missing = missing_values[missing_values > 0]

print(non_zero_missing)

# balance    25

### Задание 2

In [ ]:
# есть ли в признаке job пропущенные значения? Возможно, они обозначены каким-то специальным словом?
# ваш код
df['job'].value_counts()

# unknown            70

### Задание 3

In [ ]:
# преобразуйте признак balance таким образом, чтобы он корректно считывался, как вещественное число (float)
# ваш код
display(df['balance'].head())

# Удаляем пробелы, символы валюты и заменяем запятую на точку
#df['balance'] = df['balance'].str.replace(' ', '').str.replace('$', '').str.replace(',', '.')
df['balance'] = df['balance'].apply(lambda x: x if [x]==[np.nan] else float((x.replace(' ', '')).replace(',', '.').split('$')[0]))

# Преобразуем в float
df['balance'] = df['balance'].astype(float)

display(df['balance'].head())


In [ ]:
display(f'Среднее значение balance: {df["balance"].mean():.3f}')
# Среднее значение balance: 1529.129

### Задание 4

In [ ]:
# обработайте пропуски в признаки balance , заменив их на медианные значения по данному признаку
# ваш код
df['balance'] = df['balance'].fillna(df['balance'].median())

display(f'Среднее значение balance: {df["balance"].mean():.3f}')
# Среднее значение balance: 1526.936


### Задание 5

In [ ]:
# обработайте пропуски в категориальных признаках: job и education, заменив их на модальные значения
# ваш код

display(df['job'].value_counts())
display(df['education'].value_counts())


df['job'] = df['job'].replace('unknown', df['job'].mode()[0])
df['education'] = df['education'].replace('unknown', df['education'].mode()[0])

# Найти самую популярную работу
most_popular_job = df['job'].mode()[0]

# Найти самый популярный уровень образования
most_popular_education = df['education'].mode()[0]

# Отфильтровать клиентов с самой популярной работой и образованием
filtered_clients = df[(df['job'] == most_popular_job) & (df['education'] == most_popular_education)]

# Рассчитать средний баланс
average_balance = filtered_clients['balance'].mean()

print(f"Средний баланс для клиентов с самой популярной работой ({most_popular_job}) и образованием ({most_popular_education}):  {average_balance:.3f}")

# Средний баланс для клиентов с самой популярной работой (management) и образованием (secondary):  1598.883

### Задание 6

In [ ]:
# удалите все выбросы для признака balance

# ваш код

# Вычисляем квартили
Q1 = df['balance'].quantile(0.25)
Q3 = df['balance'].quantile(0.75)

# Рассчитываем интерквартильный размах
IQR = Q3 - Q1

# Определяем границы выбросов
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Округляем границы до целых чисел
lower_bound = round(lower_bound)
upper_bound = round(upper_bound)

print(f"Нижняя граница: {lower_bound}, Верхняя граница: {upper_bound}")

# Фильтруем данные, удаляя выбросы
df_filtered = df[(df['balance'] >= lower_bound) & (df['balance'] <= upper_bound)]

# Проверяем результат
display(df_filtered.info())
display(df_filtered.shape)

# Нижняя граница: -2241, Верхняя граница: 4063

## Часть 2:  Разведывательный анализ

### Задание 1

In [ ]:
# изучите соотношение классов в ваших данных на предмет несбалансированности, проиллюстрируйте результат
# ваш код

pio.renderers.default = "notebook" 
fig = px.histogram(df_filtered, x='deposit', title='Deposit Distribution', text_auto=True)
fig.show()



Классы относительно сбалансированы.

### Задания 2 и 3

In [ ]:
#рассчитайте описательные статистики для количественных переменных, проинтерпретируйте результат
#ваш код

# Предположим, что df - ваш DataFrame
quantitative_columns = df_filtered.select_dtypes(include=['float64', 'int64']).columns

# Получаем описательные статистики
descriptive_stats = df_filtered[quantitative_columns].describe()
display(descriptive_stats)

# Визуализация распределений с помощью Plotly
for column in quantitative_columns:
    fig = px.histogram(df, x=column, title=f'Distribution of {column}', marginal='box', nbins=30)
    fig.update_layout(bargap=0.1)
    fig.show()


На основании полученных описательных статистик для количественных переменных можно сделать следующие выводы:
------

1. Возраст (age)
Среднее значение: 40.90 года, что указывает на то, что большинство клиентов находятся в среднем возрасте.
Стандартное отклонение: 11.73, что говорит о значительной вариативности возрастов.
Минимум и максимум: Возраст варьируется от 18 до 95 лет, что охватывает широкий возрастной диапазон.
Квартильный размах: 25% клиентов моложе 32 лет, а 75% моложе 48 лет, что указывает на концентрацию клиентов в диапазоне 32-48 лет.

2. Баланс (balance)
Среднее значение: 807.65, но с высоким стандартным отклонением 994.15, что указывает на значительную вариативность.
Минимум и максимум: Баланс варьируется от -2049 до 4063, что указывает на наличие как отрицательных, так и высоких значений.
Квартильный размах: 25% клиентов имеют баланс ниже 95, а 75% ниже 1227, что указывает на асимметрию в распределении.

3. День (day)
Среднее значение: 15.59, что указывает на средний день месяца, когда происходят транзакции.
Стандартное отклонение: 8.44, что говорит о равномерном распределении транзакций в течение месяца.
Минимум и максимум: Транзакции происходят в диапазоне от 1 до 31 дня.

4. Длительность (duration)
Среднее значение: 368.74 секунд, что указывает на среднюю длительность взаимодействия.
Стандартное отклонение: 346.65, что говорит о значительной вариативности.
Минимум и максимум: Длительность варьируется от 2 до 3881 секунд, что указывает на наличие как очень коротких, так и очень длинных взаимодействий.

5. Кампания (campaign)
Среднее значение: 2.52, что указывает на среднее количество контактов в кампании.
Стандартное отклонение: 2.71, что говорит о значительной вариативности.
Минимум и максимум: Количество контактов варьируется от 1 до 43, что указывает на наличие как единичных, так и многократных контактов.

6. Дни с последнего контакта (pdays)
Среднее значение: 51.32, но с высоким стандартным отклонением 109.64, что указывает на значительную вариативность.
Минимум и максимум: Значения варьируются от -1 (никогда не контактировали) до 854 дней.
Квартильный размах: Большинство значений -1, что указывает на редкие повторные контакты.

7. Предыдущие контакты (previous)
Среднее значение: 0.82, что указывает на редкие предыдущие контакты.
Стандартное отклонение: 2.24, что говорит о значительной вариативности.
Минимум и максимум: Количество предыдущих контактов варьируется от 0 до 58.

Общие выводы:
- Вариативность: Высокое стандартное отклонение в переменных balance, duration, и pdays указывает на значительную вариативность и наличие выбросов.
- Аномалии: Наличие отрицательных значений в balance и pdays может указывать на аномалии или специфические условия данных.
- Распределение: Средние значения и медианы (50%) в некоторых переменных значительно различаются, что может указывать на асимметрию в распределении данных.

Визуализация:
- Гистограммы: Могут помочь визуально оценить распределение данных и выявить выбросы.
- Боксплоты: Полезны для выявления выбросов и оценки симметрии распределения.


### Задания 4 и 5

In [ ]:
#рассчитайте описательные статистики для категориальных переменных, проинтерпретируйте результат
#ваш код
#постройте визуализации, иллюстрирующие результаты

import math

categorical_columns = df_filtered.select_dtypes(include=['object']).columns

display(df_filtered[categorical_columns].describe())

# Получаем описательные статистики для категориальных переменных
for column in categorical_columns:
    print(f"Статистика для {column}:")
    value_counts_df = df_filtered[column].value_counts().to_frame().reset_index()
    value_counts_df.columns = [column, 'count']  # Переименовываем столбцы
    display(value_counts_df)
    print("\n")

# Определяем количество строк и столбцов для подграфиков
num_columns = len(categorical_columns)
num_rows = math.ceil(num_columns / 3)

# Визуализация распределений
plt.figure(figsize=(15, 5 * num_rows))
for i, column in enumerate(categorical_columns, 1):
    plt.subplot(num_rows, 3, i)
    sns.countplot(y=df_filtered[column], order=df_filtered[column].value_counts().index)
    plt.title(column)
    plt.xlabel('Count')
    plt.ylabel(column)

plt.tight_layout()
plt.show()

На основании предоставленной описательной статистики для категориальных переменных можно сделать следующие выводы:
------
1. Работа (job)
Уникальные значения: 11 различных категорий работы.
Наиболее частая категория: "management" с частотой 2315, что указывает на то, что большинство клиентов работают в управлении.

2. Семейное положение (marital)
Уникальные значения: 3 категории (например, "married", "single", "divorced").
Наиболее частая категория: "married" с частотой 5715, что указывает на то, что большинство клиентов состоят в браке.

3. Образование (education)
Уникальные значения: 3 категории (например, "primary", "secondary", "tertiary").
Наиболее частая категория: "secondary" с частотой 5517, что указывает на то, что большинство клиентов имеют среднее образование.

4. Наличие дефолта (default)
Уникальные значения: 2 категории ("yes", "no").
Наиболее частая категория: "no" с частотой 9939, что указывает на то, что большинство клиентов не имеют дефолта.

5. Наличие жилья (housing)
Уникальные значения: 2 категории ("yes", "no").
Наиболее частая категория: "no" с частотой 5243, что указывает на то, что у большинства клиентов нет жилья.

6. Наличие кредита (loan)
Уникальные значения: 2 категории ("yes", "no").
Наиболее частая категория: "no" с частотой 8712, что указывает на то, что у большинства клиентов нет кредита.

7. Контакт (contact)
Уникальные значения: 3 категории (например, "cellular", "telephone", "unknown").
Наиболее частая категория: "cellular" с частотой 7283, что указывает на то, что большинство контактов осуществляется через мобильные телефоны.

8. Месяц (month)
Уникальные значения: 12 месяцев.
Наиболее частая категория: "may" с частотой 2617, что указывает на то, что большинство контактов происходят в мае.

9. Исход предыдущей кампании (poutcome)
Уникальные значения: 4 категории (например, "success", "failure", "other", "unknown").
Наиболее частая категория: "unknown" с частотой 7570, что указывает на то, что для большинства клиентов исход предыдущей кампании неизвестен.

10. Депозит (deposit)
Уникальные значения: 2 категории ("yes", "no").
Наиболее частая категория: "no" с частотой 5424, что указывает на то, что большинство клиентов не открыли депозит.

**Общие выводы:**

- Дисбаланс категорий: В некоторых переменных, таких как default, housing, и loan, наблюдается значительный дисбаланс, где одна категория доминирует.
- Неизвестные значения: В переменной poutcome значительная часть данных имеет значение "unknown", что может указывать на недостаток информации или проблемы с данными.
- Преобладание определенных категорий: В переменных job, marital, и education наблюдается преобладание определенных категорий, что может быть полезно для сегментации клиентов.


### Задание 6

In [ ]:
# Узнайте, для какого статуса предыдущей маркетинговой кампании успех в текущей превалирует над количеством неудач.
# ваш код

# Сгруппируем данные по 'poutcome' и посчитаем количество значений 'deposit'
poutcome_deposit_counts = df_filtered.groupby('poutcome')['deposit'].value_counts().unstack()

# Выведем результат
display(poutcome_deposit_counts)

# Найдем статус, для которого успехов больше, чем неудач
successful_poutcome = poutcome_deposit_counts[poutcome_deposit_counts['yes'] > poutcome_deposit_counts['no']]

display("Статус предыдущей кампании, для которого успехов больше, чем неудач:")
display(successful_poutcome)

### Задание 7

In [ ]:
# узнайте, в каком месяце чаще всего отказывались от предложения открыть депозит
# ваш код

# Сгруппируем данные по 'month' и посчитаем количество отказов ('deposit' = 'no')
month_deposit_counts = df_filtered[df_filtered['deposit'] == 'no']['month'].value_counts()

# Выведем результат
display(month_deposit_counts)

# Найдем месяц с наибольшим количеством отказов
most_declined_month = month_deposit_counts.idxmax()

display(f"Месяц, в котором чаще всего отказывались от предложения открыть депозит: {most_declined_month}")

### Задание 8

In [ ]:
# создайте возрастные группы и определите, в каких группах более склонны открывать депозит, чем отказываться от предложения
# ваш код

# Определяем границы и метки для категорий
bins = [0, 30, 40, 50, 60, float('inf')]
labels = ['<30', '30-40', '40-50', '50-60', '60+']

# Преобразуем возраст в категории
df_filtered['age_group'] = pd.cut(df_filtered['age'], bins=bins, labels=labels, right=False)

# Создаем сводную таблицу для анализа
pivot_table = df_filtered.pivot_table(index='age_group', columns='deposit', aggfunc='size', fill_value=0)

# Выводим сводную таблицу
display(pivot_table)

# Построим диаграмму для визуализации различий
fig = px.histogram(
    df_filtered, 
    x='age_group', 
    color='deposit', 
    category_orders={'age_group': ['<30', '30-40', '40-50', '50-60', '60+']},
    barmode='group',
    title='Количество открытых/не открытых депозитов по возрастным группам',
    labels={'age_group': 'Возрастная группа', 'count': 'Количество'}
)

fig.update_layout(
    xaxis_title='Возрастная группа',
    yaxis_title='Количество',
    legend_title='Депозит'
)

fig.show()


### Задания 9 и 10

In [ ]:
# постройте визуализации для открывших и неоткрывших депозит в зависимости от семейного статуса
fig_marital = px.histogram(
    df_filtered, 
    x='marital', 
    color='deposit', 
    barmode='group',
    title='Количество открытых/не открытых депозитов по семейному положению',
    labels={'marital': 'Семейное положение', 'count': 'Количество'}
)

fig_marital.update_layout(
    xaxis_title='Семейное положение',
    yaxis_title='Количество',
    legend_title='Депозит'
)

fig_marital.show()

# Текстовый вывод
marital_counts = df_filtered.groupby('marital')['deposit'].value_counts().unstack()
display("Распределение депозитов по семейному положению:")
display(marital_counts)

In [ ]:
# постройте визуализации для открывших и неоткрывших депозит в зависимости от образования
fig_education = px.histogram(
    df_filtered, 
    x='education', 
    color='deposit', 
    barmode='group',
    title='Количество открытых/не открытых депозитов по уровню образования',
    labels={'education': 'Уровень образования', 'count': 'Количество'}
)

fig_education.update_layout(
    xaxis_title='Уровень образования',
    yaxis_title='Количество',
    legend_title='Депозит'
)

fig_education.show()

# Текстовый вывод
education_counts = df_filtered.groupby('education')['deposit'].value_counts().unstack()
display("Распределение депозитов по уровню образования:")
display(education_counts)

In [ ]:
# постройте визуализации для открывших и неоткрывших депозит в зависимости от вида профессиональной занятости
fig_job = px.histogram(
    df_filtered, 
    x='job', 
    color='deposit', 
    barmode='group',
    title='Количество открытых/не открытых депозитов по сфере занятости',
    labels={'job': 'Сфера занятости', 'count': 'Количество'}
)

fig_job.update_layout(
    xaxis_title='Сфера занятости',
    yaxis_title='Количество',
    legend_title='Депозит'
)

fig_job.show()

# Текстовый вывод
job_counts = df_filtered.groupby('job')['deposit'].value_counts().unstack()
display("Распределение депозитов по сфере занятости:")
display(job_counts)

### Задание 11

In [ ]:
# постройте сводную таблицу, чтобы определить люди с каким образованием и семейным статусом наиболее многочисленны
#(если рассматривать тех, кто открыл депозит)

# Разделяем таблицу на две части
df_deposit_yes = df_filtered[df_filtered['deposit'] == 'yes']
df_deposit_no = df_filtered[df_filtered['deposit'] == 'no']


# Создаем сводные таблицы
pivot_yes = df_deposit_yes.pivot_table(index='education', columns='marital', aggfunc='size', fill_value=0)
pivot_no = df_deposit_no.pivot_table(index='education', columns='marital', aggfunc='size', fill_value=0)

# Тепловая карта для тех, кто открыл депозит
plt.figure(figsize=(10, 6))
sns.heatmap(pivot_yes, annot=True, fmt="d", cmap="coolwarm")
plt.title('Тепловая карта: Открыли депозит')
plt.xlabel('Семейное положение')
plt.ylabel('Уровень образования')
plt.show()

# Находим пересечение с максимальным значением в pivot_yes
max_value = pivot_yes.max().max()
max_location = pivot_yes.stack().idxmax()

# Выводим результат в виде текста
display(f"Самое многочисленное пересечение Открывших депозит: {max_location} с количеством {max_value}")


# Тепловая карта для тех, кто не открыл депозит
plt.figure(figsize=(10, 6))
sns.heatmap(pivot_no, annot=True, fmt="d", cmap="coolwarm")
plt.title('Тепловая карта: Не открыли депозит')
plt.xlabel('Семейное положение')
plt.ylabel('Уровень образования')
plt.show()

# Находим пересечение с максимальным значением в pivot_yes
max_value_no = pivot_no.max().max()
max_location_no = pivot_no.stack().idxmax()

# Выводим результат в виде текста
display(f"Самое многочисленное пересечение Не открывших депозит: {max_location_no} с количеством {max_value_no}")



## Часть 3: преобразование данных

### Задание 1

In [ ]:
# преобразуйте уровни образования

# Создаем экземпляр LabelEncoder
label_encoder = LabelEncoder()

# Преобразуем категориальный признак 'education'
df_filtered['education'] = label_encoder.fit_transform(df_filtered['education'])

# Выводим первые несколько строк, чтобы проверить результат
display(f'Сумма значений education после преобразования LabelEncoder: {df_filtered["education"].sum()}')


In [ ]:
# Преобразуем категориальный признак 'age_group'
df_filtered['age_group'] = label_encoder.fit_transform(df_filtered['age_group'])

### Задания 2 и 3

In [ ]:
# преобразуйте бинарные переменные в представление из нулей и единиц

# Перекодируем переменную 'deposit'
df_filtered['deposit'] = df_filtered['deposit'].map({'yes': 1, 'no': 0})

# Выводим стандартное отклонение по преобразованной переменной 'deposit'
display(f"Стандартное отклонение по преобразованной переменной 'deposit': {df_filtered['deposit'].std():.3f}")

# Перекодируем переменную 'default'
df_filtered['default'] = df_filtered['default'].map({'yes': 1, 'no': 0})

# Выводим стандартное отклонение по преобразованной переменной 'default'
display(f"Стандартное отклонение по преобразованной переменной 'default': {df_filtered['default'].std():.3f}")

# Перекодируем переменную 'housing'
df_filtered['housing'] = df_filtered['housing'].map({'yes': 1, 'no': 0})

# Выводим стандартное отклонение по преобразованной переменной 'housing'
display(f"Стандартное отклонение по преобразованной переменной 'housing': {df_filtered['housing'].std():.3f}")

# Перекодируем переменную 'loan'
df_filtered['loan'] = df_filtered['loan'].map({'yes': 1, 'no': 0})

# Выводим стандартное отклонение по преобразованной переменной 'loan'
display(f"Стандартное отклонение по преобразованной переменной 'loan': {df_filtered['loan'].std():.3f}")



In [ ]:
avg_summ_encoded =  df_filtered['default'].mean()+ df_filtered['housing'].mean()+ df_filtered['loan'].mean()

display(f'Стандартное отклонение по сумме преобразованных переменных "default", "housing", "loan": {avg_summ_encoded:.3f}')
# Стандартное отклонение по сумме преобразованных переменных "default", "housing", "loan": 0.635

### Задание 4

In [ ]:
# создайте дамми-переменные

# Список номинальных переменных для преобразования
nominal_variables = ['job', 'marital', 'contact', 'month', 'poutcome']

# Создаем dummy-переменные
df_dummies = (pd.get_dummies(df_filtered[nominal_variables], drop_first=False)).astype(int)

# Добавляем dummy-переменные в исходный DataFrame
df_with_dummies = pd.concat([df_filtered, df_dummies], axis=1)

# Выводим первые несколько строк, чтобы проверить результат
display(df_with_dummies.head())
display(df_with_dummies.info())


### Задания 5 и 6

In [ ]:
# постройте корреляционную матрицу и оцените данные на предмет наличия мультиколлинеарности

numeric_columns = df_with_dummies.select_dtypes(include=['number'])

# Вычисляем матрицу корреляций для числовых столбцов
correlation_matrix = numeric_columns.corr()

# Построим тепловую карту
plt.figure(figsize=(15, 10))
sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm', fmt=".2f")
plt.title('Матрица корреляций')
plt.show()


In [ ]:
# Извлекаем корреляции с целевой переменной 'deposit_encoded'
correlation_with_target = correlation_matrix['deposit'].drop('deposit')

# Сортируем корреляции
sorted_correlation = correlation_with_target.sort_values()

# Создаем DataFrame из отсортированных коэффициентов корреляции
correlation_df = sorted_correlation.reset_index()
correlation_df.columns = ['Feature', 'Correlation']

# Построим столбчатую диаграмму с помощью Plotly
fig = px.bar(
    correlation_df, 
    x='Correlation', 
    y='Feature', 
    orientation='h', 
    title='Коэффициенты корреляции с целевой переменной',
    labels={'Correlation': 'Коэффициент корреляции', 'Feature': 'Признаки'},
    color='Correlation',
    color_continuous_scale='Blues'
)

fig.update_layout(
    xaxis_title='Коэффициент корреляции',
    yaxis_title='Признаки',
    yaxis={'categoryorder':'total ascending'}
)

fig.show()
display('Коэффициенты корреляции с целевой переменной')
display(sorted_correlation)

Мультиколлинеарность признаков отсутсвует. 

Признаки имеющие наибольшую корреляцию с целевой переменной:
- продолжительность контакта в секундах (duration)
- успех результата прошлой маркетинговой кампании (poutcome_success)
- контакт с клиентом по сотовому (contact_cellular)

### Задания 7 и 8

In [ ]:
df_with_dummies.info()

In [ ]:
df = df_with_dummies.drop(columns=['age', 'education', 'default', 'loan', 'housing'])

df = df.select_dtypes(include=['number'])
#df = df_with_dummies.copy()

X = df.drop(['deposit'], axis=1)
y = df['deposit']
 
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state = 42, test_size = 0.33)



In [ ]:
# рассчитайте необходимые показатели

# Размеры выборки
display(f'Размер тренировочной выборки X_train={X_train.shape}, y_train={y_train.shape[0]}')
display(f'Размер тестовой выборки X_test={X_test.shape}, y_test={y_test.shape[0]}')

# Вычисляем среднее значение целевой переменной на тестовой выборке
display(f"Среднее значение целевой переменной на тестовой выборке: {y_test.mean():.2f}")

# 'Размер тренировочной выборки X_train=(6770, 39), y_train=6770'
# 'Размер тестовой выборки X_test=(3335, 39), y_test=3335'
# 'Среднее значение целевой переменной на тестовой выборке: 0.46'

### Задание 9

In [ ]:
# с помощью SelectKBest отберите 15 наиболее подходящих признаков

selector = SelectKBest(score_func=f_classif, k=15)
X_train_selected = selector.fit_transform(X_train, y_train)

# Получаем имена отобранных признаков
selected_features = X_train.columns[selector.get_support()]

selector.get_feature_names_out()
selector_col = list(selector.get_feature_names_out())

# Выводим отобранные признаки
display("Отобранные признаки:", selector_col)



### Задание 10

In [ ]:
# нормализуйте данные с помощью minmaxsxaler
from sklearn.preprocessing import MinMaxScaler

# Создаем объект MinMaxScaler
scaler = MinMaxScaler()

# Применяем нормализацию к обучающей выборке
X_train_scaled = scaler.fit_transform(X_train)
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)

# Применяем нормализацию к тестовой выборке
X_test_scaled = scaler.transform(X_test)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

# Выводим первые несколько строк нормализованных данных для проверки
display("Нормализованные данные обучающей выборки:", X_train_scaled[:5])
display("Нормализованные данные тестовой выборки:", X_test_scaled[:5])


In [ ]:
# Рассчитайте среднее арифметическое для первого предиктора (т. е. для первого столбца матрицы) из тестовой выборки. 
# Ответ округлите до двух знаков после точки-разделителя.

mean_first_predictor = X_test_scaled.iloc[:, 0].mean()
display(f'Среднее арифметическое для первого предиктора {mean_first_predictor:.2f}')
# 0.47 должно быть

# Часть 4: Решение задачи классификации: логистическая регрессия и решающие деревья

### Задание 1

In [ ]:
# обучите логистическую регрессию и рассчитайте метрики качества
log_model = linear_model.LogisticRegression(random_state=42, solver='sag', max_iter=1000)
log_model.fit(X_train_scaled, y_train)
y_test_pred = log_model.predict(X_test_scaled)
y_train_pred = log_model.predict(X_train_scaled)
print(f'Мерики на тестовых даннных\n {metrics.classification_report(y_test, y_test_pred)}')
print(f'Мерики на тренировочных данных\n {metrics.classification_report(y_train, y_train_pred)}')

### Задания 2,3,4

In [ ]:
# обучите решающие деревья, настройте максимальную глубину

tree_model = tree.DecisionTreeClassifier(random_state=42, criterion='entropy')
tree_model.fit(X_train_scaled, y_train)
y_test_pred = tree_model.predict(X_test_scaled)
y_train_pred = tree_model.predict(X_train_scaled)
print(f'Мерики на тестовых даннных\n {metrics.classification_report(y_test, y_test_pred)}')
print(f'Мерики на тренировочных данных\n {metrics.classification_report(y_train, y_train_pred)}')


In [ ]:
from sklearn import tree, metrics
import numpy as np

# Диапазон значений для максимальной глубины дерева
max_depths = range(1, 7)

# Списки для хранения точности на обучающей и тестовой выборках
train_accuracies = []
test_accuracies = []

# Перебираем различные значения максимальной глубины
for max_depth in max_depths:
    # Создаем и обучаем модель дерева решений
    tree_model = tree.DecisionTreeClassifier(random_state=42, criterion='entropy', max_depth=max_depth)
    tree_model.fit(X_train_scaled, y_train)
    
    # Предсказания на обучающей и тестовой выборках
    y_train_pred = tree_model.predict(X_train_scaled)
    y_test_pred = tree_model.predict(X_test_scaled)
    
    # Вычисляем точность
    train_accuracy = metrics.accuracy_score(y_train, y_train_pred)
    test_accuracy = metrics.accuracy_score(y_test, y_test_pred)
    
    # Сохраняем точность
    train_accuracies.append(train_accuracy)
    test_accuracies.append(test_accuracy)

# Находим максимальную глубину, при которой точность на тестовой выборке максимальна, но не наблюдается переобучения
optimal_depth = max_depths[np.argmax(test_accuracies)]

# Выводим результаты
print(f'Оптимальная глубина дерева: {optimal_depth}')
print(f'Точность на обучающей выборке: {train_accuracies[optimal_depth-1]:.3f}')
print(f'Точность на тестовой выборке: {test_accuracies[optimal_depth-1]:.3f}')


tree_model = tree.DecisionTreeClassifier(random_state=42, criterion='entropy', max_depth=optimal_depth)
tree_model.fit(X_train_scaled, y_train)
y_test_pred = tree_model.predict(X_test_scaled)
y_train_pred = tree_model.predict(X_train_scaled)
print(f'Мерики на тестовых даннных с оптимальной глубиной\n {metrics.classification_report(y_test, y_test_pred)}')
print(f'Мерики на тренировочных данных с оптимальной глубиной\n {metrics.classification_report(y_train, y_train_pred)}')

In [ ]:
oo = pd.DataFrame([tree_model.feature_importances_], columns=X_train_scaled.columns)
fig = px.bar(x = list(oo.loc[0].sort_values(ascending=False)[0:10].index),
    y=round(oo.loc[0].sort_values(ascending=False)[0:10], 2),
    text_auto=True,
    title='ТОП-10 признаков'    
)
fig.show()

### Задание 5

In [ ]:
# подберите оптимальные параметры с помощью gridsearch
from sklearn.model_selection import GridSearchCV
# задаём сетку параметров
param_grid = [
              {'min_samples_split': [2, 5, 7, 10] , # Минимальное количество выборок, необходимое для разделения внутреннего узла
              'max_depth':[3,5,7] # Максимальная глубина дерева
               }              
]

grid_search = GridSearchCV(
    estimator=tree.DecisionTreeClassifier(criterion='entropy',
        random_state=42 #генератор случайных чисел
        ), 
    param_grid=param_grid, 
    n_jobs = -1,
    scoring='f1'
)  

grid_search.fit(X_train_scaled, y_train) 
print(f'Best Hyperparameter Values: {grid_search.best_params_}')
print(f'Best Models:{grid_search.best_estimator_}')
print(f'Best score Cross validation: {grid_search.best_score_:.3f}')
y_test_pred = grid_search.predict(X_test_scaled)
y_train_pred = grid_search.predict(X_train_scaled)
print(f'Мерики на тестовых даннных\n {metrics.classification_report(y_test, y_test_pred)}')
print(f'Мерики на тренировочных данных\n {metrics.classification_report(y_train, y_train_pred)}')

# Часть 5: Решение задачи классификации: ансамбли моделей и построение прогноза

### Задание 1

In [ ]:
# обучите на ваших данных случайный лес
# from sklearn.ensemble import RandomForestClassifier
# from sklearn import metrics

# Создаем и обучаем модель случайного леса
rf_model = RandomForestClassifier(
    n_estimators=100,
    criterion='gini',
    min_samples_leaf=5,
    max_depth=10,
    random_state=42
)

# Обучаем модель на обучающей выборке
rf_model.fit(X_train_scaled, y_train)

# Предсказания на тестовой и обучающей выборках
y_test_pred = rf_model.predict(X_test_scaled)
y_train_pred = rf_model.predict(X_train_scaled)

# Выводим метрики на тестовых данных
print(f'Метрики на тестовых данных\n{metrics.classification_report(y_test, y_test_pred)}')

# Выводим метрики на обучающих данных
print(f'Метрики на обучающих данных\n{metrics.classification_report(y_train, y_train_pred)}')

# accuracy 0.83
# recall 0.84


### Задания 2 и 3

In [ ]:
# используйте для классификации градиентный бустинг и сравните качество со случайным лесом

# Создаем и обучаем модель градиентного бустинга
gb_model = GradientBoostingClassifier(
    learning_rate=0.05,
    n_estimators=300,
    min_samples_leaf=5,
    max_depth=5,
    random_state=42
)

# Обучаем модель на обучающей выборке
gb_model.fit(X_train_scaled, y_train)

# Предсказания на тестовой и обучающей выборках
y_test_pred = gb_model.predict(X_test_scaled)
y_train_pred = gb_model.predict(X_train_scaled)

# Выводим метрики на тестовых данных
print(f'Метрики на тестовых данных\n{metrics.classification_report(y_test, y_test_pred)}')

# Выводим метрики на обучающих данных
print(f'Метрики на обучающих данных\n{metrics.classification_report(y_train, y_train_pred)}')

#F1 - 0.82

### Задание 4

In [ ]:
# объедините уже известные вам алгоритмы с помощью стекинга 

# Определяем базовые модели
decision_tree = DecisionTreeClassifier(random_state=42, criterion='entropy', max_depth=10)
logistic_regression = LogisticRegression(random_state=42, max_iter=1000)
gradient_boosting = GradientBoostingClassifier(
    learning_rate=0.05,
    n_estimators=300,
    min_samples_leaf=5,
    max_depth=5,
    random_state=42
)

# Определяем метамодель
meta_model = LogisticRegression(random_state=42, max_iter=1000)

# Создаем стекинг-классификатор
stacking_model = StackingClassifier(
    estimators=[
        ('decision_tree', decision_tree),
        ('logistic_regression', logistic_regression),
        ('gradient_boosting', gradient_boosting)
    ],
    final_estimator=meta_model,
    cv=5
)

# Обучаем стекинг-модель на обучающей выборке
stacking_model.fit(X_train_scaled, y_train)

# Предсказания на тестовой и обучающей выборках
y_test_pred = stacking_model.predict(X_test_scaled)
y_train_pred = stacking_model.predict(X_train_scaled)

# Выводим метрики на тестовых данных
print(f'Метрики на тестовых данных\n{metrics.classification_report(y_test, y_test_pred)}')

# Выводим метрики на обучающих данных
print(f'Метрики на обучающих данных\n{metrics.classification_report(y_train, y_train_pred)}')

# precition = 0.82

### Задание 5

In [ ]:
# оцените, какие признаки демонстрируют наибольшую  важность в модели градиентного бустинга

# Получаем важности признаков
feature_importances = gb_model.feature_importances_

# Создаем DataFrame для удобства отображения
importance_df = pd.DataFrame({
    'Feature': X_train_scaled.columns,
    'Importance': feature_importances
})

# Сортируем по важности
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Выводим топ-10 признаков по важности
print("Топ-10 признаков по важности в модели градиентного бустинга:")
print(importance_df.head(10))

### Задания 6,7,8

In [ ]:
# реализуйте оптимизацию гиперпараметров с помощью Optuna
%%time
# Функция для оптимизации
def objective(trial):
    # Определяем гиперпараметры для оптимизации
    n_estimators = trial.suggest_int('n_estimators', 100, 200, 1)
    max_depth = trial.suggest_int('max_depth', 10, 30, 1)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 10, 1)
    
    # Создаем модель случайного леса с текущими гиперпараметрами
    rf_model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_leaf=min_samples_leaf,
        random_state=42
    )
    
    # Оцениваем модель с помощью кросс-валидации
    score = cross_val_score(rf_model, X_train_scaled, y_train, cv=5, scoring=make_scorer(f1_score)).mean()
    
    return score

# Создаем исследование Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# Выводим лучшие гиперпараметры и лучший результат
print(f'Лучшие гиперпараметры: {study.best_params}')
print(f'Лучший F1-score: {study.best_value:.3f}')

# рассчитаем точность для тестовой выборки
model = ensemble.RandomForestClassifier(**study.best_params,random_state=42, )
model.fit(X_train_scaled, y_train)
y_train_pred = model.predict(X_train_scaled)
print("accuracy на тестовом наборе: {:.2f}".format(model.score(X_test_scaled, y_test)))
y_test_pred = model.predict(X_test_scaled)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

# accuracy = 0.83
# f1 = 0.84

Выводы
=====

1. Предобработка данных:

- Данные были успешно подготовлены для моделирования. Категориальные переменные были преобразованы в числовые с использованием dummy-переменных и LabelEncoder.
- Применена min-max нормализация к предикторам, что позволило улучшить сходимость моделей и их производительность.

2. Анализ признаков:

- Проведен анализ важности признаков с использованием модели градиентного бустинга. Наиболее важные признаки были выявлены, что может помочь в дальнейшем улучшении модели и интерпретации результатов.
- Матрица корреляций показала, что некоторые признаки имеют высокую корреляцию с целевой переменной, что может быть полезно для дальнейшего анализа.

3. Моделирование:

- Были обучены и оценены несколько моделей, включая решающие деревья, случайный лес и градиентный бустинг. Каждая модель показала свои сильные и слабые стороны.
- Стекинг, объединяющий несколько моделей, продемонстрировал улучшение качества предсказаний, что подтверждает эффективность использования ансамблевых методов.

4. Оптимизация гиперпараметров:

- Использование GridSearchCV и Optuna для оптимизации гиперпараметров позволило улучшить производительность моделей. Optuna, в частности, показал себя как мощный инструмент для автоматизированного поиска оптимальных параметров.

5. Результаты и рекомендации:

- Наилучшие результаты были достигнуты с использованием модели градиентного бустинга и стекинга, что указывает на их высокую эффективность для данной задачи.
- Рекомендуется продолжить исследование и оптимизацию моделей, возможно, с использованием дополнительных данных или более сложных архитектур, чтобы еще больше улучшить качество предсказаний.